## THE CIRCUIT CONFIGURATION

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Set the current working directory
import os
os.chdir('/content/drive/MyDrive/ml-verifier-oracle/prover')
print("Current path: " + os.getcwd())

Mounted at /content/drive
Current path: /content/drive/MyDrive/ml-verifier-oracle/prover


In [2]:
# check if notebook is in colab
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import json
import ezkl
import pandas as pd

import torch
from torchvision import transforms
from PIL import Image

import logging
import os
import sys
import sys
import time
from datetime import datetime

current_dir = os.getcwd()
#project_root = os.path.abspath(os.path.join(current_dir, '..'))
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)
from training.defineSNN import MODSiameseBambooNN
from training.defineSNN2 import FurtherAdjustedSiameseBambooNN


# uncomment for more descriptive logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.INFO)


# Specify all the files we need
model_path = os.path.join('ezkl-outputs/snn.onnx')
compiled_model_path = os.path.join('ezkl-outputs/snn.ezkl')
pk_path = os.path.join('ezkl-outputs/pk.key')
vk_path = os.path.join('ezkl-outputs/vk.key')
settings_path = os.path.join('ezkl-outputs/settings.json')
srs_path = os.path.join('ezkl-outputs/kzg.srs')
data_path = os.path.join('ezkl-outputs/input.json')
cal_path = os.path.join('ezkl-outputs/cal_data.json')
##
sol_code_path = os.path.join('../contracts/Verifier.sol')
abi_path = os.path.join('ezkl-outputs/Verifier.abi')
proof_path = os.path.join('ezkl-outputs/proof.json')




In [3]:
# Check the ezkl version installed (>=7.1.12 has RUST log with all errors report)
!pip list | grep ezkl

ezkl                             7.1.12


In [4]:
# Assign benchmark test name each time a new test is executed to save its results into a file

timestamp = datetime.now().strftime("%m_%d_%H-%M")
test_name = f"test_011___{timestamp}"
test_time_and_size_path = os.path.join(f'../benchmark/{test_name}_time_and_size.txt')

print(test_name)

test_011___01_22_15-25


In [5]:
# Load the image pair as input ready for inference
class SiameseImageLoader:
    def __init__(self, transform=None):
        self.transform = transform

    def load_and_transform_pair(self, image1_path, image2_path):
        img1 = Image.open(image1_path).convert('L')
        img2 = Image.open(image2_path).convert('L')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        img1 = img1.to(device)
        img2 = img2.to(device)

        return img1, img2

transform = transforms.Compose([
    transforms.Resize((10, 10)),
    transforms.ToTensor(),
])

# Example inputs (TrueNegative, TruePositive, FalseNegative)
# (FalseNegative were produced with a growing rate of bamboo higher than what the model was trained on)
siamese_loader = SiameseImageLoader(transform)
image1_path = "TestValid_t1.jpeg"
image2_path = "TestValid_t0.jpeg"
img1, img2 = siamese_loader.load_and_transform_pair(image1_path, image2_path) # Separate tensors for each image
img1 = img1.unsqueeze(0)
img2 = img2.unsqueeze(0)

print(img1.size())

torch.Size([1, 1, 10, 10])


In [9]:
os.getcwd()

'/content/drive/MyDrive/ml-verifier-oracle/prover'

In [11]:
# Load the trained nn model and make inference on test inputs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = FurtherAdjustedSiameseBambooNN().to(device)
loaded_model.load_state_dict(torch.load('../training/trained_simplesnn_lr_0.01.pth', map_location=torch.device('cpu')))
loaded_model.eval()

with torch.no_grad():
    finalout = loaded_model(img1, img2)
    predicted_label = (torch.sigmoid(finalout) > 0.5).item()

print(f'Value of model outcome before sigmoid: {finalout}')
print(f'Same class probability: {torch.sigmoid(finalout)}')
print(f'Predicted Label: {predicted_label}')

# Calculate the total number of parameters
total_params = sum(p.numel() for p in loaded_model.parameters())
print(f'Total Parameters: {total_params}')

Value of model outcome before sigmoid: tensor([[7.8228]], device='cuda:0')
Same class probability: tensor([[0.9996]], device='cuda:0')
Predicted Label: True
Total Parameters: 69921


In [12]:
# Export the loaded Siamese Network model to ONNX
torch.onnx.export(loaded_model,                      # loaded Siamese Network model
                  (img1, img2),                      # model input (or a tuple for multiple inputs)
                  model_path,                        # where to save the ONNX model
                  export_params=True,                # store the trained parameter weights inside the model file
                  opset_version=14,                  # the ONNX version to export the model to
                  do_constant_folding=True,          # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})



# Move tensors from GPU to CPU
img1_cpu = img1.cpu()
img2_cpu = img2.cpu()
print(img1_cpu.size())

# Serialize data into a JSON file
input_data = [
    img1_cpu.detach().numpy().reshape([-1]).tolist(),
    img2_cpu.detach().numpy().reshape([-1]).tolist(),
]

data = dict(input_data=input_data)
json.dump(data, open(data_path, 'w'))

torch.Size([1, 1, 10, 10])


In [13]:
# *** Providing the calibrate settings function with a larger and broader range of sample inputs.

path_to_dataset = '../training/dataset10k20x20'

csv_path = os.path.join(path_to_dataset, 'forest_dataset.csv')
df = pd.read_csv(csv_path)

# Construct the full paths to the images
img1_files = [os.path.join('..', 'training/dataset10k20x20', path) for path in df['imageT0'].iloc[:20]]
img2_files = [os.path.join('..', 'training/dataset10k20x20', path) for path in df['imageT1'].iloc[:20]]

img1_list = []
img2_list = []
for img1_path, img2_path in zip(img1_files, img2_files):
    img1, img2 = siamese_loader.load_and_transform_pair(img1_path, img2_path)
    img1 = img1.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img2 = img2.unsqueeze(0).cpu().detach().numpy().reshape([-1]).tolist()
    img1_list.extend(img1) # don't use .append bc it should look like [[xxx...],[yyy...]]
    img2_list.extend(img2)

data = dict(input_data = [img1_list, img2_list])
# Serialize data into file:
json.dump( data, open(cal_path, 'w' ))

In [14]:
#For ezkl to compute a snark, it needs some settings to determine how to create the circuit.
#This cell instantiates some parameters that determine the circuit shape, size etc
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # "fixed" for params means that the committed to params are used for all proofs
py_run_args.variables = [("batch_size", 1)]

!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True
print("gen_settings OK")

# *** SCALES is one of main knobs you can turn to trade off accuracy for proving efficiency
# Under the hood calibration iterates over the scales array to see how precise you can go before failure.
# For example if 7 fails it falls back to 1.
# (Right now the default scales values for the resources target is 8-10 and 10-13  for accuracy)
res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources", scales = [1, 8])
#res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "accuracy")

INFO:ezkl.graph.model:set batch_size to 1
INFO:ezkl.graph.model:model has 1 instances
INFO:ezkl.graph.model:calculating num of constraints using dummy model layout...
INFO:ezkl.graph.model:model uses 39888 rows (coord=79777, constants=76111)
INFO:ezkl.graph.model:set batch_size to 1
INFO:ezkl.execute:num of calibration batches: 20
INFO:ezkl.execute:running onnx predictions...
INFO:ezkl.graph.model:set batch_size to 1
INFO:tract_linalg.x86_64_fma:qmmm_i32: x86_64/avx2 activated
INFO:tract_linalg.x86_64_fma:mmm_f32, mmv_f32, sigmoid_f32, tanh_f32: x86_64/fma activated


gen_settings OK


INFO:ezkl.graph.model:set batch_size to 1
INFO:ezkl.graph.model:model has 1 instances
INFO:ezkl.graph.model:calculating num of constraints using dummy model layout...
INFO:ezkl.graph.model:model uses 6862 rows (coord=13725, constants=9893)
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph:input scales: [1, 1]
INFO:ezkl.graph.mode

In [15]:
# Compile the model into a circuit
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

INFO:ezkl.graph.model:set batch_size to 1


In [16]:
# Get public srs from kzg ceremony.
res = ezkl.get_srs(settings_path)
assert res == True

INFO:ezkl.execute:SRS does not exist, downloading...
INFO:ezkl.execute:SRS downloaded
INFO:ezkl.execute:SRS hash: c09129f064c08ecb07ea3689a2247dcc177de6837e7d2f5f946e30453abbccef


In [17]:
# Setup the circuit and make sure the keys are generated afterwards.
start_time = time.time()

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
    )

end_time = time.time()

execution_time = end_time - start_time

vk_size = os.path.getsize(vk_path)
pk_size = os.path.getsize(pk_path)

with open(test_time_and_size_path, 'a') as f:
   f.write(f"Verification key size: {vk_size} bytes\n")
   f.write(f"Proving key size: {pk_size} bytes\n")
   f.write(f"Setup time: {execution_time}\n")

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)

INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg14.srs"
INFO:ezkl.execute:downsizing params to 14 logrows
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.graph.model:model layout...
INFO:ezkl.graph.model:model uses 6331 rows (coord=12662, constants=8904)
INFO:ezkl.pfsys:VK took 6.208
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.graph.model:model layout...
INFO:ezkl.graph.model:model uses 6331 rows (coord=12662, constants=8904)
INFO:ezkl.pfsys:PK took 1.933
INFO:ezkl.pfsys:saving verification key 💾
INFO:ezkl.pfsys:saving proving key 💾


In [18]:
# Install solidity compiler
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])
    !solc-select install 0.8.20
    !solc-select use 0.8.20
    !solc --version
# rely on local installation if the notebook is not in colab
except:
    pass


Installing solc '0.8.20'...
Version '0.8.20' installed.
Switched global version to 0.8.20
solc, the solidity compiler commandline interface
Version: 0.8.20+commit.a1b79de6.Linux.g++


In [19]:
# Generate the verifier associated with the circuit
res = ezkl.create_evm_verifier(
        vk_path,
        settings_path,
        sol_code_path,
        abi_path
    )
assert res == True
assert os.path.isfile(sol_code_path)

INFO:ezkl.execute:checking solc installation..
INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg14.srs"
INFO:ezkl.execute:downsizing params to 14 logrows
INFO:ezkl.pfsys:loading verification key from "ezkl-outputs/vk.key"
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}


## PROVE and VERIFY
### here we will generate and verify a proof locally. Then we will format the inputs and the proof in a way compatible for the evm verifier

In [20]:
witness_path = os.path.join('ezkl-outputs/witness.json')

# generate the witness file
res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

INFO:ezkl.graph:input scales: [1, 1]


In [21]:
# Generate the proof
start_time = time.time()

proof = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )

end_time = time.time()

execution_time = end_time - start_time
with open(test_time_and_size_path, 'a') as f:
   f.write(f"Proving time: {execution_time}\n")

print(proof)
assert os.path.isfile(proof_path)

INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg14.srs"
INFO:ezkl.execute:downsizing params to 14 logrows
INFO:ezkl.pfsys:loading proving key from "ezkl-outputs/pk.key"
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.pfsys:proof started...
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.graph.model:model layout...
INFO:ezkl.graph.model:model uses 6331 rows (coord=12662, constants=8904)
INFO:ezkl.pfsys:proof took 16.400


{'instances': [[[15320674181704580433, 2068178628828129834, 6959445960512757249, 2508920288600862730]]], 'proof': '0x09b8ba2fcb283551c49f1cb7545e237cc62de70b5fdf5e9d2e236f80d9fd0e490c7f87c2508706a3ce91d6a9227126b0f6872b400d269e8e59a2e403435289bd09bb55373ac29d3708f4dd19291f49b1ac7f2f6b8f6b4f79cca57b088da432a505dfffac06a96aa3af7dd607b692ab56c41b975ee34be796717729295097e7c3298ee074a96b6404dcfb9c1b57fd8316e43121bf71f421070d3181a689b2c1c72c1b79faf419271df4fff36051e300f54b6ec8a3597aacb59f1d11a5f3510ba5135d18644ba639f1f314eacf452659777f659efba8fcce2e371179f4d9e76d031e31ddfe4b638f4234b8a58e69ba7eec7a2f411ac95209c5e0b05df7566cbeaa14800a4a77fea4923e04d61d7ec379e6a7fb7db79830ffabc7c5632e85775cb500a9c92ecec8d5f4486aa1516e67792dda74fcabed610b8a3767930465c7926c032e58b456a2ed83c2b0a7e96fcf110b015ec94f13dea16f18b89e4b754bf2631c26f960e945c99fc01fe2a85a28cc24add283bcee54c62542ee76e699bbaa992dc19c410c9cec28b1b7f1a6089817777b0fb6307453b17a66b92b2927b7a191036a13d7d87b57d2fc74c8ca4f7116d6b21fe4a1252f40ba4e8

In [22]:
# Sanity check off-chain verification
start_time = time.time()

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

end_time = time.time()

execution_time = end_time - start_time
with open(test_time_and_size_path, 'a') as f:
   f.write(f"Verification (locally) time: {execution_time}\n")

assert res == True
print("verified locally")

INFO:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg14.srs"
INFO:ezkl.execute:downsizing params to 14 logrows
INFO:ezkl.pfsys:loading verification key from "ezkl-outputs/vk.key"
INFO:ezkl.graph.vars:number of blinding factors: 5
INFO:ezkl.graph.model:configuring model
INFO:ezkl.graph:circuit size: 
 {
  "num_advice_columns": 6,
  "num_challenges": 0,
  "num_fixed": 4,
  "num_instances": 1,
  "num_selectors": 24
}
INFO:ezkl.execute:verify took 0.20
INFO:ezkl.execute:verified: true


verified locally


In [23]:
# Get input data for the verifier contract (evmInputs.json)
onchain_input_array = []
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(proof["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"
print("form output",formatted_output )

input_list = [int(entry, 0) for entry in formatted_output.strip('[]').split(', ')]
quoted_list = [format(entry, "#066x") for entry in input_list]
json_data = {
    "instances": quoted_list,
    "proof": proof["proof"]
}
with open('evmInputs.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=2)
print("Saved evmInputs.json: ", json_data)


form output [0x0000000000000000000000000000000000000000000000000000000000000082]
Saved evmInputs.json:  {'instances': ['0x0000000000000000000000000000000000000000000000000000000000000082'], 'proof': '0x09b8ba2fcb283551c49f1cb7545e237cc62de70b5fdf5e9d2e236f80d9fd0e490c7f87c2508706a3ce91d6a9227126b0f6872b400d269e8e59a2e403435289bd09bb55373ac29d3708f4dd19291f49b1ac7f2f6b8f6b4f79cca57b088da432a505dfffac06a96aa3af7dd607b692ab56c41b975ee34be796717729295097e7c3298ee074a96b6404dcfb9c1b57fd8316e43121bf71f421070d3181a689b2c1c72c1b79faf419271df4fff36051e300f54b6ec8a3597aacb59f1d11a5f3510ba5135d18644ba639f1f314eacf452659777f659efba8fcce2e371179f4d9e76d031e31ddfe4b638f4234b8a58e69ba7eec7a2f411ac95209c5e0b05df7566cbeaa14800a4a77fea4923e04d61d7ec379e6a7fb7db79830ffabc7c5632e85775cb500a9c92ecec8d5f4486aa1516e67792dda74fcabed610b8a3767930465c7926c032e58b456a2ed83c2b0a7e96fcf110b015ec94f13dea16f18b89e4b754bf2631c26f960e945c99fc01fe2a85a28cc24add283bcee54c62542ee76e699bbaa992dc19c410c9cec28b1b7f1a6089817

In [24]:
# Add a description to txt test file
description = "Test SNN with total number of parameters: 69921\nNumber of samples in dataset: 10000"
with open(test_time_and_size_path, 'a') as f:
   f.write(f"\n" + description)

In [25]:
# Open and read the content of the file
with open(test_time_and_size_path, 'r') as file:
    file_content = file.read()

# Print the content
print(file_content)

Verification key size: 50632 bytes
Proving key size: 78694144 bytes
Setup time: 15.315864086151123
Proving time: 16.898720026016235
Verification (locally) time: 0.0889284610748291

Test SNN with total number of parameters: 69921
Number of samples in dataset: 10000
